In [17]:
!pip install playwright nest_asyncio bs4 requests python-dateutil
!playwright install

import nest_asyncio
nest_asyncio.apply()


Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.12/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [18]:
!pip install fake-useragent


In [4]:
!pip install playwright
!playwright install chromium
!pip install cloudscraper


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 5.8 MB/s eta 0:00:00


In [1]:
import os
import shutil

target_dir = "/content/forms"

if os.path.exists(target_dir):
    shutil.rmtree(target_dir)
    print(f"Deleted the entire contents of the folder: {target_dir}")
else:
    print(f"Directory does not exist: {target_dir}")


os.makedirs(target_dir, exist_ok=True)
print(f"Recreated empty folder: {target_dir}")

Deleted the entire contents of the folder: /content/forms
Recreated empty folder: /content/forms


In [2]:
import requests
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime
from urllib.parse import urljoin, urlparse
import csv

# Cài đặt Cloudscraper
try:
    import cloudscraper
    scraper = cloudscraper.create_scraper()
    USE_CLOUD_SCRAPER = True
except ImportError:
    USE_CLOUD_SCRAPER = False

# ============================ CONFIG ============================

BASE_URLS = [
    "https://thuviennhadat.vn/phap-luat/huong-dan-dien-don-cung-cap-du-lieu-dat-dai-tu-1-7-2025-mau-don-cung-cap-du-lieu-dat-dai-mau-so-14-m-691127.html",
    "https://luatsubaoho.com/phapluat/mau-don-dang-ky-bien-dong-dat-dai-co-huong-dan-cach-viet/",
    "https://thuvienphapluat.vn/hoi-dap-phap-luat/mau-don-yeu-cau-thi-hanh-an-dan-su-moi-nhat-2024-138012162.html",
    "https://ebh.vn/bao-hiem-y-te/mau-giay-chung-nhan-khong-cung-chi-tra-bhyt-trong-nam",
    "https://vppbentre.vn/san-pham/in-hoa-don-ban-le-theo-yeu-cau/",
    "https://azoka.vn/in-hoa-don-ban-le?srsltid=AfmBOorpex3zoh_mfNU_GXiwjjOvdOGA-CU30ABzUtkQjXkkKgzOIl1s",
    "https://banhangexcel.com/mau-phieu-mua-hang-quan-ly-noi-bo-bang-excel/",
    "https://thuvienphapluat.vn/hoi-dap-phap-luat/tai-mau-hoa-don-ban-hang-ban-word-excel-pdf-moi-nhat-2024-va-huong-dan-cach-viet-138025514.html",
    "https://onetechgroup.com.vn/mau-hoa-don-ban-hang",
    "https://luatsubaoho.com/phapluat/mau-don-dang-ky-ma-so-ma-vach/",
    "https://thuviennhadat.vn/phap-luat/huong-dan-dien-don-dang-ky-bien-dong-dat-dai-tu-1-7-2025-mau-don-dang-ky-bien-dong-dat-dai-mau-so-15-690449.html",
    "https://thuvienphapluat.vn/hoi-dap-phap-luat/mau-m01-mau-phieu-dang-ky-du-tuyen-vao-lop-10-ha-noi-2025-138046539.html",
    "https://thuvienphapluat.vn/phap-luat/ho-tro-phap-luat/mau-phieu-dang-ky-du-thi-tot-nghiep-thpt-2024-pdf-do-bo-giao-duc-ban-hanh-co-dang-nhu-the-nao-tai-m-244651-151055.html",
    "https://bizzi.vn/tai-mau-don-dat-hang-excel-chuan-nhat-mien-phi/",
    "https://thuvienphapluat.vn/banan/tin-tuc/tong-hop-mau-don-xin-viec-moi-nhat-va-huong-dan-cach-viet-11482",
    "https://www.topcv.vn/mau-don-xin-viec",
    "https://xaydungso.vn/download/tai-ngay-99-mau-don-hang-shopee-pdf-mau-don-tien-loi-nhat-vi-cb.html#12",

]

DOWNLOAD_DIR = "forms"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)
CSV_FILE = "downloaded_files.csv"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept-Language": "vi,en;q=0.9"
}

DB_DATE = datetime(2024, 1, 1)


CRITICAL_KEYWORDS = ["mẫu", "đơn", "biểu mẫu", "tờ khai", "don", "phieu", "phieu dang ky", "mau", "bieu mau", "phieu-dang-ky", "mau-bien-ban", "bang-ke-to-khai", "bang-ke", "to-khai"]
# Secondary keywords (file, download, download) are still used to find potential links.
ALL_KEYWORDS = CRITICAL_KEYWORDS + ["file", "tải", "download", ".doc", ".pdf", ".jpg", ".png", ".xls", ".xlsx"]

# ============================ CSV STORAGE FUNCTION ============================
def save_csv_row(page_title, file_url, file_name, file_extension, page_date):
    header = ["Tieu_de_trang", "Link_file", "Ten_file", "Dang_tep", "Ngay_dang"]
    write_header = not os.path.exists(CSV_FILE)
    date_str = page_date.strftime("%Y-%m-%d") if page_date else ""

    with open(CSV_FILE, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(header)
        writer.writerow([page_title, file_url, file_name, file_extension, date_str])

# ============================ DATE SPLIT FUNCTION  ============================

DATE_PATTERNS = [
    r"(Thứ Hai|Thứ Ba|Thứ Tư|Thứ Năm|Thứ Sáu|Thứ Bảy|Chủ Nhật)[,]\s*(\d{1,2}/\d{1,2}/\d{4})",
    r"(\d{1,2}/\d{1,2}/\d{4})",
    r"(\d{4}-\d{1,2}-\d{1,2})",
]

def parse_date_str(date_str):
    date_str = date_str.strip()
    date_str = re.sub(r"(Thứ Hai|Thứ Ba|Thứ Tư|Thứ Năm|Thứ Sáu|Thứ Bảy|Chủ Nhật)[,]\s*", "", date_str, flags=re.IGNORECASE).strip()

    try:
        if "/" in date_str:
            dt = datetime.strptime(date_str, "%d/%m/%Y")
        elif "-" in date_str:
            dt = datetime.strptime(date_str, "%Y-%m-%d")
        else:
            return None

        if dt.year >= 2000:
            return dt
        else:
            return None
    except:
        return None

def extract_date(html_text):
    if not html_text:
        return None

    valid_dates = []
    for p in DATE_PATTERNS:
        matches = re.findall(p, html_text, flags=re.IGNORECASE)
        for match in matches:
            if isinstance(match, tuple):
                date_str = match[-1]
            else:
                date_str = match

            dt = parse_date_str(date_str)
            if dt:
                valid_dates.append(dt)

    if valid_dates:
        return max(valid_dates)

    return None

# ============================ FILE DOWNLOAD FUNCTION ============================
def download_file(url, page_title, page_date):
    exts = [".pdf", ".doc", ".docx", ".jpg", ".png", ".xls", ".xlsx"]

    url_base = url.split("?")[0]
    filename = url_base.split("/")[-1]
    filename = re.sub(r'[\\/:*?"<>|]', '_', filename)

    file_ext = next((ext for ext in exts if filename.lower().endswith(ext)), None)

    if not file_ext or not filename:
        return False, None

    try:
        if USE_CLOUD_SCRAPER:
            r = scraper.get(url, headers=HEADERS, timeout=15)
        else:
            r = requests.get(url, headers=HEADERS, timeout=15)

        r.raise_for_status()

        file_path = os.path.join(DOWNLOAD_DIR, filename)
        with open(file_path, "wb") as f:
            f.write(r.content)

        print(f"     Downloaded successfully: {filename}")

        save_csv_row(page_title, url, filename, file_ext, page_date)

        return True, filename
    except Exception as e:
        return False, None

# ============================ FUNCTION TO GET LINK ============================
def extract_form_links(url):
    print(f"\n - Scanning the page: {url}")

    try:
        if USE_CLOUD_SCRAPER:
            r = scraper.get(url, headers=HEADERS, timeout=15)
        else:
            r = requests.get(url, headers=HEADERS, timeout=15)
        r.raise_for_status()
    except:
        print("    ⚠️ Cannot open page:", url)
        return [], None, None

    soup = BeautifulSoup(r.text, "html.parser")
    page_title = soup.find("title")
    page_title = page_title.get_text(strip=True) if page_title else "No Title"

    page_date = extract_date(r.text)
    if not page_date:
        print("    ⛔ No post date found → SKIP this page")
        return [], None, page_title

    print(f"    📅 Date discovered on page: {page_date}")

    links = []
    file_exts = [".pdf", ".doc", ".docx", ".jpg", ".png", ".xls", ".xlsx"]

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith(("javascript:", "mailto:", "tel:", "#")):
            continue

        href = urljoin(url, href)
        text = a.get_text(strip=True).lower()

        # NEW FILTER CONDITIONS:
        is_file_link = any(href.lower().endswith(ext) for ext in file_exts)

        # Check important keywords
        has_critical_keyword = any(k in text or k in href.lower() for k in CRITICAL_KEYWORDS)

        # Accept link if:
        # 1. Is the file link AND contains important keywords (make sure it is a form)
        if is_file_link and has_critical_keyword:
             links.append(href)
             continue

        # 2. NOT a file link but contains common keywords (is a link to a child page that needs to be crawled)
        if not is_file_link and any(k in text or k in href.lower() for k in ALL_KEYWORDS):
            links.append(href)

    return list(set(links)), page_date, page_title


# ============================ MAIN CRAWLER ============================
def crawl_forms():
    total_downloaded = 0

    for base_url in BASE_URLS:
        print(f"\n==============================")
        print(f"🌐 START SCANNING: {base_url}")
        print(f"==============================")

        level1_links, date1, title1 = extract_form_links(base_url)

        if not date1 or date1 <= DB_DATE:
            print(f"⛔ Level 1 pages are old or have no dates ({date1}) → skip all")
            continue

        for link in level1_links:
            # DOWNLOAD FILES DIRECTLY AT LEVEL 1
            if any(link.lower().endswith(ext) for ext in [".pdf", ".doc", ".docx", ".jpg", ".png", ".xls", ".xlsx"]):
                is_downloaded, file_name = download_file(link, title1, date1)
                if is_downloaded:
                    total_downloaded += 1
                continue

            # CRAWL LEVEL 2 (Subpage link)
            try:
                if urlparse(link).netloc != urlparse(base_url).netloc:
                    continue

                sub_links, date2, title2 = extract_form_links(link)

                if not date2 or date2 <= DB_DATE:
                    print(f"    ⛔ Old/undated secondary page ({date2}) → skip:", link)
                    continue

                for sub in sub_links:
                    # DOWNLOAD FILES AT LEVEL 2
                    if any(sub.lower().endswith(ext) for ext in [".pdf", ".doc", ".docx", ".jpg", ".png", ".xls", ".xlsx"]):
                        is_downloaded, file_name = download_file(sub, title2, date2)
                        if is_downloaded:
                            total_downloaded += 1
            except Exception as e:
                continue

    print(f"\n COMPLETED — Total number of files downloaded: {total_downloaded}")
    print(f" Statistical data is stored at: {os.path.abspath(CSV_FILE)}")

# ======= RUN =======
if __name__ == "__main__":
    crawl_forms()


🌐 START SCANNING: https://thuviennhadat.vn/phap-luat/huong-dan-dien-don-cung-cap-du-lieu-dat-dai-tu-1-7-2025-mau-don-cung-cap-du-lieu-dat-dai-mau-so-14-m-691127.html

 - Scanning the page: https://thuviennhadat.vn/phap-luat/huong-dan-dien-don-cung-cap-du-lieu-dat-dai-tu-1-7-2025-mau-don-cung-cap-du-lieu-dat-dai-mau-so-14-m-691127.html
    📅 Date discovered on page: 2025-11-01 00:00:00

 - Scanning the page: https://thuviennhadat.vn/du-an-bat-dong-san
    📅 Date discovered on page: 2025-11-01 00:00:00

 - Scanning the page: https://thuviennhadat.vn/bat-dong-san/quoc-lo-62-di-qua-cac-xa-phuong-nao-tinh-tay-ninh-sau-sap-nhap-26379.html
    📅 Date discovered on page: 2025-11-01 00:00:00

 - Scanning the page: https://thuviennhadat.vn/phap-luat/du-an-khu-dan-cu-tan-thinh-thanh-hoa-tai-phuong-dong-son-tinh-thanh-hoa-co-dien-tich-quy-hoach-la-ba-715357.html
    📅 Date discovered on page: 2025-11-01 00:00:00

 - Scanning the page: https://thuviennhadat.vn/ban-bat-dong-san-khac-toan-quoc
    📅